In [ ]:
!pip install keras-preprocessing
import os
import nltk
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import re
from keras_preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 1.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/NLP project 2/hotel_data.csv')

In [ ]:
data.head()

,area,city,country,crawl_date,highlight_value,hotel_overview,hotel_star_rating,image_urls,in_your_room,is_value_plus,...,property_type,qts,query_time_stamp,room_types,site_review_count,site_review_rating,sitename,state,traveller_rating,uniq_id
0,Hardasji Ki Magri,Udaipur,India,2016-06-21,{{facility}},|Zion Home Stay is located in a city that sets...,1 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,4.5,makemytrip,NaN,NaN,78ddf880bd7937d384ff278cc5b39d6e
1,Near Nai Gaon,Udaipur,India,2016-06-21,{{facility}},| Araliayas Resorts is a 3 star hotel located ...,3 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,4.5,makemytrip,NaN,NaN,9f9f9cbb2f7df8089b63d5cdeb257944
2,Near Bagore Ki Haveli,Udaipur,India,2016-06-21,{{facility}},|A 2 star property is located at 24 km from Ma...,2 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,NaN,makemytrip,NaN,NaN,b314bb7fa8bfb1ed306f517be21d729e
3,Dabok,Udaipur,India,2016-06-21,Airport Transfer|Car rental|Conference Hall|Cu...,|SNP House Airport Hotel And Restaurant is loa...,1 star,//imghtlak.mmtcdn.com/images/hotels/2014071815...,NaN,no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,`standard,/5,NaN,makemytrip,NaN,NaN,e6f5bb3c2d76a78d978b9ceb0e31ec56
4,East Udaipur,Udaipur,India,2016-06-21,{{facility}},| Hotel Pichola Haveli is situated in the beau...,2 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,3.7,makemytrip,NaN,NaN,63072c301427b6ca450d31eea127bcf0


In [ ]:
data.city.value_counts()

NewDelhiAndNCR    1163
Goa               1122
Mumbai             543
Jaipur             534
Bangalore          512
                  ... 
Kollur               1
Madla                1
Jeypore              1
Jispa                1
Haldia               1
Name: city, Length: 770, dtype: int64

In [ ]:
data.shape

(20000, 33)

In [ ]:
array=['Mumbai']
data=data.loc[data['city'].isin(array)]

In [ ]:
data.head()

,area,city,country,crawl_date,highlight_value,hotel_overview,hotel_star_rating,image_urls,in_your_room,is_value_plus,...,property_type,qts,query_time_stamp,room_types,site_review_count,site_review_rating,sitename,state,traveller_rating,uniq_id
294,Charai,Mumbai,India,2016-08-28,Doctor on Call|Front desk|Laundry Service|Park...,"Nestled in Mumbai, a city with strong historic...",3,NaN,Bathroom Toiletries|Attached Bathroom|Hot & Co...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Ac Superior Room|A/c Standard Rooms Double Occ...,NaN,NaN,makemytrip,Maharashtra,NaN,d78fae90ef2e1b5c2dfd547c61763a25
309,Andheri (East),Mumbai,India,2016-08-28,Air Conditioned|Airport Transfer|Conference Ha...,3 km from Chhatrapati Shivaji International Ai...,2,NaN,Bathroom Toiletries|Daily Newspaper|Kitchenett...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Executive|Deluxe,NaN,NaN,makemytrip,Maharasta,Location:3.2/5 | Hospitality:3.1/5 | Facilitie...,030865f741982beb373efddecdc6d6c3
321,Khar,Mumbai,India,2016-08-28,Airport/Rlwy Stn Transfer|Bar|Conference Hall|...,Location Hotel Royal Garden is situated on Juh...,3,NaN,Electronic Safe|Bathroom Toiletries|Daily News...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Superior Executive,NaN,NaN,makemytrip,Maharashtra,Location:4.5/5 | Hospitality:3.4/5 | Facilitie...,a1ced509350038775a7700ec67796bc2
334,Andheri (East),Mumbai,India,2016-08-28,24 Hour Check in-Icon|24 hour reception|24 hou...,City Guest House is a beautiful property locat...,2,NaN,Bathroom Toiletries|Hot/cold Water|Attached Ba...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Standard Room|Deluxe Room|Triple Deluxe Room|S...,NaN,NaN,makemytrip,Maharashtra,Location:4.3/5 | Hospitality:3.8/5 | Facilitie...,f2820ae7707668ed6906bb227921f720
1238,Andheri (East),Mumbai,NaN,2016-08-22,24 Hour Check in-Icon|24 hour reception|24 hou...,Sai Residency Hotel is situated in the City of...,2,NaN,Bathroom Toiletries|Attached Bathroom|Hot & Co...,no,...,Hotel,2016-08-22 22:10:53 +0000,2016-08-22 22:10:53 +0000,Deluxe Dbl Air Cooled,NaN,NaN,makemytrip,MAHARASHTRA,NaN,b4af24952027ffbcd85a91bb6fe23f5d


In [ ]:
data=data.hotel_overview

In [ ]:
data.isna().sum()

53

In [ ]:
data.dropna(inplace=True)
data.isna().sum()

0

In [ ]:
data.iloc[0]

'Nestled in Mumbai, a city with strong historical links, wonderful British architecture, museums, beaches, places of worship, and above all, a true galaxy of stars where Bollywood reigns supreme. The Hotel Divya International is a delightful leisure with an absolute blend of service, charm and efficiency. The hotel offers facilities like front desk, parking, laundry, doctor on call and many more. It aims to extend the best possible hospitality experience to all its revered customers. The hotel is located at a distance of 350 m from S.T Workshop Bus Stop, 22.5 km from Chhatrapati Shivaji International Airport and 37.1 km from Chhatrapati Shivaji Terminus. The guests can head to prominent tourist attractions like Sanjay Gandhi National Park, KidZania Mumbai, Haji Ali Mosque, Iskon Temple, Shree Siddhivinayak Temple and many more. Shoppers can have fun filled shopping experience shopping at Viviana Mall(1.7 km). Food lovers can visit nearby restaurants to try local delights. The hotel has

In [ ]:
nltk.download('stopwords')
stop=set(stopwords.words('english'))
print(stop)

{'while', 'am', "haven't", 'them', 'your', 'ma', 'didn', 'll', 'just', 'y', 'being', "wouldn't", "you'll", 'haven', 'more', 'myself', 've', 'what', "isn't", 'under', 'me', 'against', 'can', 'into', 'hers', "you're", 'she', 'too', 'm', 'by', 'same', 'you', 'should', 'ain', "should've", 'wasn', 'between', 'a', 'up', 'or', 'whom', 'there', "that'll", 'him', 'again', 'nor', 'each', 'about', 'i', 'had', 'these', 'but', 'only', 'other', 'because', 'than', "don't", 'an', 'having', 'itself', "didn't", 'are', 'off', 'will', 'doing', "shan't", 'that', 'very', 'did', 'we', 's', 'all', 'once', 'both', 'of', 'is', 'no', 'to', 'down', "aren't", 'our', 'own', 'here', 'it', 'when', 'its', "won't", 'weren', 'with', 'doesn', 'd', 'their', 'few', 'have', 'if', "weren't", 'my', 'then', 'wouldn', 'needn', 'won', 'this', 'he', 'ourselves', 'after', 'the', 'further', 'hasn', 'before', "needn't", 'does', 'couldn', "mightn't", 'yours', 'out', 'over', 'why', 'do', 'not', 'any', 'herself', 'they', "hadn't", 'at'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data.head()

294     Nestled in Mumbai, a city with strong historic...
309     3 km from Chhatrapati Shivaji International Ai...
321     Location Hotel Royal Garden is situated on Juh...
334     City Guest House is a beautiful property locat...
1238    Sai Residency Hotel is situated in the City of...
Name: hotel_overview, dtype: object

In [ ]:
def stopwords_removal(data_point):
    data=[x for x in data_point.split() if x not in stop]
    return data

In [ ]:
def clean_data(data):
    cleaned_data=[]
    all_unique_words_in_each_description=[]
    for entry in data:
        entry = re.sub(pattern='[^a-zA-Z]',repl=' ',string = entry)
        entry = re.sub(r'\b\w{0,1}\b', repl=' ',string = entry)
        entry = entry.lower()
        entry = stopwords_removal(entry)
        cleaned_data.append(entry)
        unique = list(set(entry))
        all_unique_words_in_each_description.extend(unique)
    return cleaned_data, all_unique_words_in_each_description

In [ ]:
def unique_words(data):
    unique_words = set(data)
    return unique_words, len(unique_words)

In [ ]:
cleaned_data, all_unique_words_in_each_description = clean_data(data)
unique_words, length_of_unique_words = unique_words(all_unique_words_in_each_description)

In [ ]:
print(cleaned_data[0])

['nestled', 'mumbai', 'city', 'strong', 'historical', 'links', 'wonderful', 'british', 'architecture', 'museums', 'beaches', 'places', 'worship', 'true', 'galaxy', 'stars', 'bollywood', 'reigns', 'supreme', 'hotel', 'divya', 'international', 'delightful', 'leisure', 'absolute', 'blend', 'service', 'charm', 'efficiency', 'hotel', 'offers', 'facilities', 'like', 'front', 'desk', 'parking', 'laundry', 'doctor', 'call', 'many', 'aims', 'extend', 'best', 'possible', 'hospitality', 'experience', 'revered', 'customers', 'hotel', 'located', 'distance', 'workshop', 'bus', 'stop', 'km', 'chhatrapati', 'shivaji', 'international', 'airport', 'km', 'chhatrapati', 'shivaji', 'terminus', 'guests', 'head', 'prominent', 'tourist', 'attractions', 'like', 'sanjay', 'gandhi', 'national', 'park', 'kidzania', 'mumbai', 'haji', 'ali', 'mosque', 'iskon', 'temple', 'shree', 'siddhivinayak', 'temple', 'many', 'shoppers', 'fun', 'filled', 'shopping', 'experience', 'shopping', 'viviana', 'mall', 'km', 'food', 'lo

In [ ]:
length_of_unique_words

3395

In [ ]:
def build_indices(unique_words):
    word_to_idx = {}
    idx_to_word = {}
    for i, word in enumerate(unique_words):
        word_to_idx[word] = i
        idx_to_word[i] = word
    return word_to_idx, idx_to_word

In [ ]:
word_to_idx, idx_to_word = build_indices(unique_words)
print(idx_to_word)

{0: 'calls', 1: 'swanky', 2: 'life', 3: 'bites', 4: 'phone', 5: 'marol', 6: 'conscious', 7: 'facilitate', 8: 'plays', 9: 'order', 10: 'mangatram', 11: 'chosen', 12: 'dreamhome', 13: 'fly', 14: 'arrange', 15: 'sure', 16: 'kalyan', 17: 'festooned', 18: 'celestial', 19: 'emanates', 20: 'fancy', 21: 'purifier', 22: 'varied', 23: 'resto', 24: 'chinchoti', 25: 'huge', 26: 'visualizers', 27: 'storage', 28: 'delights', 29: 'rejuvinating', 30: 'flat', 31: 'partition', 32: 'sauna', 33: 'tuskers', 34: 'splurge', 35: 'le', 36: 'birla', 37: 'sana', 38: 'cheerful', 39: 'historic', 40: 'talve', 41: 'select', 42: 'paris', 43: 'shalimar', 44: 'mix', 45: 'immaculately', 46: 'revered', 47: 'venues', 48: 'sofa', 49: 'sufficient', 50: 'session', 51: 'closes', 52: 'named', 53: 'arranging', 54: 'wintergreen', 55: 'lite', 56: 'choosing', 57: 'heritage', 58: 'offering', 59: 'th', 60: 'kamgar', 61: 'wood', 62: 'unwind', 63: 'day', 64: 'events', 65: 'grills', 66: 'plunge', 67: 'phoenix', 68: 'cadell', 69: 'greet

In [ ]:
def prepare_corpus(corpus, word_to_idx):
    
    sequences = []
    for line in corpus:
        tokens = line
        for i in range(1, len(tokens)):
            i_gram_sequence = tokens[:i+1]
            i_gram_sequence_ids = []
            
            for j, token in enumerate(i_gram_sequence):
                i_gram_sequence_ids.append(word_to_idx[token])
                
            sequences.append(i_gram_sequence_ids)
    
    return sequences

In [ ]:
sequences = prepare_corpus(cleaned_data, word_to_idx)
max_sequence_len = max([len(x) for x in sequences])

In [ ]:
max_sequence_len

308

In [ ]:
print(sequences[0])
print(sequences[1])

[2954, 1659]
[2954, 1659, 1863]


In [ ]:
print(idx_to_word[1647])
print(idx_to_word[867])
print(idx_to_word[1452])

may
worry
baroke


In [ ]:
len(sequences)

51836

In [ ]:
def build_input_data(sequences, max_sequence_len, length_of_unique_words):
    sequences = np.array(pad_sequences(sequences, maxlen = max_sequence_len, padding = 'pre'))
    X = sequences[:,:-1]
    y = sequences[:,-1]
    y = np_utils.to_categorical(y, length_of_unique_words)
    return X, y

In [ ]:
length_of_unique_words

3395

In [ ]:
X, y = build_input_data(sequences, max_sequence_len, length_of_unique_words)

In [ ]:
def create_model(max_sequence_len, length_of_unique_words):
    model = Sequential()
    model.add(Embedding(length_of_unique_words, 10, input_length=max_sequence_len - 1))
    model.add(LSTM(128))
    model.add(Dropout(0.2))
    model.add(Dense(length_of_unique_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [ ]:
model = create_model(max_sequence_len, length_of_unique_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 307, 10)           33950     
                                                                 
 lstm (LSTM)                 (None, 128)               71168     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 3395)              437955    
                                                                 
Total params: 543,073
Trainable params: 543,073
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X, y, batch_size = 512, epochs=200)

Epoch 1/200
102/102 [==============================] - 5s 46ms/step - loss: 2.5986
Epoch 2/200
102/102 [==============================] - 5s 48ms/step - loss: 2.5876
Epoch 3/200
102/102 [==============================] - 5s 46ms/step - loss: 2.5762
Epoch 4/200
102/102 [==============================] - 5s 45ms/step - loss: 2.5658
Epoch 5/200
102/102 [==============================] - 5s 50ms/step - loss: 2.5402
Epoch 6/200
102/102 [==============================] - 5s 45ms/step - loss: 2.5298
Epoch 7/200
102/102 [==============================] - 5s 45ms/step - loss: 2.5158
Epoch 8/200
102/102 [==============================] - 5s 49ms/step - loss: 2.4997
Epoch 9/200
102/102 [==============================] - 5s 45ms/step - loss: 2.4856
Epoch 10/200
102/102 [==============================] - 5s 48ms/step - loss: 2.4760
Epoch 11/200
102/102 [==============================] - 5s 47ms/step - loss: 2.4829
Epoch 12/200
102/102 [==============================] - 5s 47ms/step - loss: 2.4637
E

In [ ]:



save_dir = "/content/drive/MyDrive/NLP project 2/"


if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# we had saved the model architecture in a JSON file
model_structure = model.to_json()
with open(os.path.join(save_dir, "my_model.json"), "w") as json_file:
    json_file.write(model_structure)

# model with h5 file
model.save_weights(os.path.join(save_dir, "my_model_weights.h5"))

In [ ]:
def generate_text(seed_text, next_words, model, max_seq_len):
    for _ in range(next_words):
        cleaned_data = clean_data([seed_text])
        sequences= prepare_corpus(cleaned_data[0], word_to_idx)
        sequences = pad_sequences([sequences[-1]], maxlen=max_seq_len-1, padding='pre')
        predicted_x = model.predict(sequences, verbose=0)
        predicted=np.argmax(predicted_x,axis=1)
        output_word = ''
        output_word = idx_to_word[predicted[0]]            
        seed_text = seed_text + " " + output_word
        
    return seed_text.title()

In [ ]:
print(generate_text("in Mumbai there we need", 30, model, max_sequence_len))

In Mumbai There We Need Mumbai Hotel Located Mumbai Km Sahar International Airport Km Sion Railway Station Km Bus Stand Hotel Offers Facilities Like Front Desk Room Service Room Service Laundry Service Doctor Call Laundry


In [66]:
print(generate_text("Best Mumbai", 30, model, max_sequence_len))

ValueError: ignored